## Renaming the Unregistered Students: 

In [8]:
unknown = 'Unknown'
unknown += input('Enter the Unique ID of the Student whose name should be re-written: ')
known = input('Enter Actual Name: ')

Enter the Unique ID of the Student whose name should be re-written: 1
Enter Actual Name: 20BCE2667


In [10]:
import numpy as np
faces = np.load('encoding.npy',allow_pickle='TRUE').item()
value = faces[unknown]
del faces[unknown]
faces[known] = value
np.save('encoding.npy', faces)

KeyError: 'Unknown1'

## Face Recognition 

In [11]:
import face_recognition
import cv2
import numpy as np
from skimage.io import imread
import os

In [12]:
# import numpy as np

# count = 0
# faces = np.load('encoding.npy',allow_pickle='TRUE').item()
# known_face_names = list(faces.keys())
# if known_face_names[-1][:7] == 'Unknown':
#     count = int(known_face_names[-1][7:])
# count

file = open("global_count.txt", "r")
count = int(file.read())
file.close()
count

1

In [13]:
def encoding(name1):
    #Load
    faces = np.load('encoding.npy',allow_pickle='TRUE').item()

    encoded = {}
    face = imread('load-wb/'+ name1 +'.jpg')                      # Without Box - wb
    encoding = face_recognition.face_encodings(face)[0]
    #print(type(encoding))
    encoded[name1] = encoding
    faces.update(encoded)

    # Save
    np.save('encoding.npy', faces)

In [14]:
video_capture = cv2.VideoCapture(0)

def classify_face():
    global_count = count
#     faces = get_encoded_faces()
#     known_face_encodings = list(faces.values())
#     known_face_names = list(faces.keys())

    # Load
    faces = np.load('encoding.npy',allow_pickle='TRUE').item()
    
    known_face_encodings = list(faces.values())
    known_face_names = list(faces.keys())
    
    # Initialize some variable
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    nameList = []

    while True:
        ret, frame = video_capture.read()
        
        small_frame = cv2.resize(frame, (0,0), None, fx=0.25, fy=0.25)

        rgb_small_frame = small_frame#[:,:,::-1]
        
        #face_locations = face_recognition.face_locations(rgb_small_frame, model='hog')
        face_locations = face_recognition.face_locations(rgb_small_frame, model='cnn')
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        name ="Not Detected"
        
        for encodeFace,faceLoc in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_face_encodings,encodeFace,0.5)
            faceDis = face_recognition.face_distance(known_face_encodings, encodeFace)
            matchIndex = np.argmin(faceDis)
            
            
            #if True in matches:
            if matches[matchIndex]:
                matchIndex = matches.index(True)
                name = known_face_names[matchIndex]
                
            elif (False == (True in matches)):
                global_count += 1
                file = open("global_count.txt", "w")
                file.write(str(global_count))
                file.close()
                name ="Unknown"+str(global_count)
                y1,x2,y2,x1 = faceLoc
                y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
                cframe = frame[y1-10:y2+10, x1-10:x2+10]  #[x1:x2, y1:y2]  # Cropped Frame - image[start_x:end_x, start_y:end_y]
                cv2.imwrite('load-wb/'+ name +'.jpg', cframe)
                print(name)
                encoding(name)
                faces = np.load('encoding.npy',allow_pickle='TRUE').item()
                known_face_encodings = list(faces.values())
                known_face_names = list(faces.keys())
                # print(x1,x2,y1,y2) # print(frame.shape)
                
            
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0),2)
            cv2.rectangle(frame,(x1,y2-35), (x2,y2), (0,255,0), cv2.FILLED)
            cv2.putText(frame, name, (x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),2)
            cframe = frame[y1-10:y2+10, x1-10:x2+10]  #[x1:x2, y1:y2]  # Cropped Frame - image[start_x:end_x, start_y:end_y]
            cv2.imwrite('load/'+ name +'.jpg', cframe)
            nameList.append(name)
            print(name,end="")
            print()
            #for i, faceDis in enumerate(faceDis):
                #cv2.putText(frame, str(round(faceDis,2)), (x2-55,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),2 )
            #cv2.putText(frame, str(round(faceDis[1],2)), (x2-40,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),1)
            
        if name=="Not Detected":
            cv2.putText(frame, name, (5,50),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
        
        # Display Result
        cv2.imshow("Video", frame)
        cv2.imwrite('DetectedFaces/faces_detected.jpg', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            #print(str(round(faceDis,2)))
            print(name)
            break
        
        if name != "Not Detected":
            if name != 'Unknown':
                pass
                #print(str(round(faceDis,2)))
#                 print(name)################################
#                 cv2.waitKey(0)
#                 video_capture.release()
#                 cv2.destroyAllWindows()
#                 nameList.append(name)##############################
#                 return name
            else:
                pass
#                 print(name)
#                 cframe = frame[x1:x2, y1:y2]  # Cropped Frame
#                 cv2.imwrite('load-wb/'+ name +'.jpg', cframe)
                
#                 cv2.imwrite('load-wb/'+ name +'.jpg', frame)
#                 cv2.waitKey(0)
#             break
    
    file = open("global_count.txt", "w")
    file.write(str(global_count))
    file.close() 
    
    video_capture.release()
    cv2.destroyAllWindows()
    return nameList

In [15]:
Detected_face = classify_face()

20BCE2667
20BCE2667
20BCE2667
20BCE2667
20BCE2667
Unknown2
Unknown2
Unknown2
20BCE2667
20BCE2667
20BCE2667
20BCE2667


In [6]:
f = list(set(Detected_face))
f.sort()
f

['Unknown1']

In [10]:
import csv

with open('DATA.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        #print(row[0])
        if row[0] == Detected_face:
            print(row)

['Abhinav Jaiswal', '20BCE2624', '8957058660', 'AB+']


In [22]:
f = list(set(Detected_face))
f.sort()
f

['20BCE2667', 'Unknown3']

In [16]:
import pandas as pd
df = pd.read_csv('Attendance.csv')
# df = df.sort_values(by=['Reg.No.'])
# j = 0
# for i in range(len(df)):
#     df['Status'][i] = 'Absent'
c= 0 
for i in f:
    c = 0
    for j in range(len(df)):
        if i == df['Reg.No.'][j]:
            c+=1
    if c!=1:
        new_row = {'Reg.No.':i}
        df = df.append(new_row, ignore_index=True)
#     if i in df['Reg.No.']:
        
#         print(i)
#         new_row = {'Reg.No.':i}
#         df = df.append(new_row, ignore_index=True)
# print(df)      
for j in range(len(f)):
    for i in range(len(df)):
#         print(df['Reg.No.'][j], f[i])
        if df['Reg.No.'][i] == f[j]:
            df['Status'][i] = 'Present'
            break
for i in range(len(df)):
    if df['Status'][i] != 'Present':
        df['Status'][i] = 'Absent'

df

C:\Users\vansh\AppData\Local\Temp/ipykernel_4636/554560784.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status'][i] = 'Present'
C:\Users\vansh\AppData\Local\Temp/ipykernel_4636/554560784.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status'][i] = 'Absent'


,Unnamed: 0,Name,Reg.No.,Status
0,0.0,Vanshit Kandoi,20BCE2667,Present
1,1.0,Abhinav Jaiswal,20BCE2624,Absent
2,2.0,Sneha Jayshri,20BCE2673,Absent
3,3.0,Ben Afflek,ABC1,Absent
4,4.0,Jerry Seinfeld,ABC2,Absent
5,5.0,Elton John,ABC3,Absent
6,6.0,Madonna,ABC4,Absent
7,7.0,Mindy Kaling,ABC5,Absent
8,8.0,Govinda,XYZ1,Absent
9,9.0,Tabu,XYZ2,Absent


In [25]:
df.to_csv('Attendance.csv')